In [65]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import bs4 as bs

In [66]:
#get an url list 
def get_url_list(page1_url):
    var_url = '&section_offset=5&items_offset='
    url_lst = [page1_url]
    for page in range(1,5):
        url = page1_url+var_url+str(18*page)
        url_lst.append(url)
    return url_lst

In [67]:
def get_house_info(url, price):
    #url = 'https://www.airbnb.com/rooms/3503946?s=P8Cyps_7'
    import requests
    from bs4 import BeautifulSoup
    import re
    response = requests.get(url)
    output = []
    if not response.status_code == 200:
        print('failed to get the fidelity website')
        return output
    else:
        results_page = BeautifulSoup(response.content,'html5lib')
        # rating
        if len(results_page.find_all('div', class_="_l0ao8q")) == 0:
            rating = 'no rating'
        else:
            rating_sec = results_page.find_all('div', class_="_l0ao8q")[0]
            rating = rating_sec.find('span').get('aria-label')
        # coordinates
        coordinates = re.search(
            '"listing_lat":([0-9]{2}.[0-9]*),"listing_lng":(-[0-9]{2}.[0-9]*),', 
            response.text)
        
        lat = coordinates.group(1)
        lon = coordinates.group(2)
        # host
        superhost = re.search('"is_superhost":(.{1,5})', 
                              response.text).group(1)
        
        if len(results_page.find_all('span', class_="_fgdupie"))==0:
            response_rate = "no info"
        else:
            host = results_page.find_all('span', class_="_fgdupie")
            #response rate
            response_rate = float('nan')
            for host_ in host[::-1]:
                if 'Response rate' in host_.text:
                    response_rate = host_.text.split(':')[-1]
                    break
        #accomondates      
        if re.search('"guest_label":".{1,8}([0-9][0-9]?).{1,8}",', response.text) != None:
            accommodates = re.search('"guest_label":".{1,8}([0-9][0-9]?).{1,8}",', response.text).group(1)
        else:
            accommodates = re.search('"guest_label":"([0-9][0-9]?) guest.*', response.text).group(1)

            #number of bedrooms
        if re.search('"bedroom_label":"([0-9][0-9]?).*","guest_label"', response.text) != None:
            bedrooms = re.search('"bedroom_label":"([0-9][0-9]?).*","guest_label"', response.text).group(1)
        else:
            bedrooms= '0'
            #number of beds
        beds = re.search('"bed_label":"(.).*","bedroom_label"', response.text).group(1)       
            #number of bathrooms
        bathrooms = re.search('"bathroom_label":"[0-9].?[0-9]? (.*)","bed_label"', response.text).group(1)
        
        
        #room type
        if re.search('"bedroom_label":"[0-9] (.*)","guest_label"', response.text) != None:
            room_type = re.search('"bedroom_label":"[0-9] (.*)","guest_label"', response.text).group(1)
        else:
            room_type= re.search('"bedroom_label":"(..?.?.?.?.?.?.?.?.?.?.?)","guest_label"', response.text).group(1)

        #number of review
        try:
            number_of_reviews = results_page.find('div', class_="_vy3ibx").text
        except AttributeError:
            number_of_reviews='NoneType'
        
        output=[price, accommodates, bedrooms,beds, bathrooms, room_type, number_of_reviews,  rating, lat, lon, superhost, response_rate]
        #print('output=')
        #print(output)
        
    return output

In [60]:
def get_prices_urls(url,dataframe):
    info_list=[]
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    browser = webdriver.Chrome('/Users/yutingzhang/Desktop/ops_con/chromedriver')
    #url='https://www.airbnb.com/s/Manhattan--New-York--NY--United-States/homes?refinement_paths%5B%5D=%2Fhomes&toddlers=0&query=Manhattan%2C%20New%20York%2C%20NY%2C%20United%20States&allow_override%5B%5D=&price_min=27&price_max=43&s_tag=qfon6p2A'
    browser.get(url)
    html_source = browser.page_source
    results_page = BeautifulSoup(html_source, "lxml")
    #get the prices for every apartment since the price infomation is not availabe inside the apt page
    #prices = results_page.find_all('span',class_='_pd52isb')
    prices=browser.find_elements(By.XPATH,"//span[@class='_17oldnte']")
    price_list=[element.text for element in prices][::3]

    #get the url for every apt on this page
    urls = results_page.find_all("meta", content=re.compile("www.airbnb.com/rooms"))
    url_list = []
          
    for url in urls:
        url = str(url)
        url = "https://"+url[15:-18]
        #print (x)
        url_list.append(url)
    print(len(url_list))
     #pass url and price to the get_house_info function and extract the rest infomation 
    for x in range(len(price_list)):
        #print(get_house_info(url_list[x], price_list[x]))
        info_list.append(get_house_info(url_list[x], price_list[x]))
    df2=pd.DataFrame(info_list,columns=['price','accommodates', 'bedrooms','beds', 'bathrooms', 'room_type', 'number_of_reviews',  'rating', 'lat', 'lon', 'superhost', 'response_rate'])
    dataframe=pd.concat([dataframe,df2],ignore_index=True)

    #print("info_list=")
    #print(info_list)
    return dataframe  

In [61]:
page_url='https://www.airbnb.com/s/homes?refinement_paths%5B%5D=%2Fhomes&query=manhattan&toddlers=0&price_min=40&price_max=86&allow_override%5B%5D=&s_tag=Ix7KHU6j'
url_list=get_url_list(page_url)
df = pd.DataFrame(columns=['price','accommodates', 'bedrooms','beds', 'bathrooms', 'room_type', 'number_of_reviews',  'rating', 'lat', 'lon', 'superhost', 'response_rate'])
for url in url_list:
    #update df
    df=get_prices_urls(url,df)

16
0
16
18
0


IndexError: list index out of range

In [62]:
df

,price,accommodates,bedrooms,beds,bathrooms,room_type,number_of_reviews,rating,lat,lon,superhost,response_rate
0,Price\n$80 per night,4,1,2,private bath,bedroom,97 Reviews,Rating 4.5 out of 5,40.722234641605674,-73.99526741088673,false,98%
1,Price\n$49 per night,2,1,1,shared bath,bedroom,554 Reviews,Rating 4.5 out of 5,40.82264063496292,-73.940408848972,false,100%
2,Price\n$75 per night,2,1,1,shared bath,bedroom,283 Reviews,Rating 5 out of 5,40.72648794825359,-73.97903515407235,"true,",97%
3,Price\n$63 per night,2,1,1,private bath,bedroom,93 Reviews,Rating 4.5 out of 5,40.79273872227299,-73.94561750449053,false,90%
4,Price\n$65 per night,2,1,1,private bath,bedroom,230 Reviews,Rating 4.5 out of 5,40.79774208163206,-73.9324393794276,"true,",94%
5,Price\n$84 per night,3,0,1,bath,Studio,45 Reviews,Rating 4.5 out of 5,40.71553042533876,-73.99117817625118,false,100%
6,Price\n$59 per night,2,1,1,bath,bedroom,264 Reviews,Rating 4.5 out of 5,40.788825741051674,-73.97616147567851,false,100%
7,Price\n$79 per night,3,0,2,bath,Studio,65 Reviews,Rating 4 out of 5,40.73878992564724,-73.99771916899928,false,100%
8,Price\n$70 per night,2,1,1,shared bath,bedroom,428 Reviews,Rating 4.5 out of 5,40.74524432585626,-74.00319819551403,false,100%
9,Price\n$56 per night,2,1,1,private bath,bedroom,145 Reviews,Rating 4.5 out of 5,40.766254465749675,-73.99387298516461,"true,",100%


In [64]:
len(df['lat'].unique())

50

In [33]:
df.to_csv('/Users/yutingzhang/Desktop/40-86df_16.csv')